In [2]:
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
import numpy as np

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F

def initiateSession():
    connection_parameters = {
        "account": "mszvisn-web31075",
        "user": "jenkins_srv",
        "password": "Jenkins123",
        "role": "accountadmin",
        "warehouse": "compute_wh",
        "database":"snowpark_db",
        "schema":"cat_bronze"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

Transforming data from Bronze layer to Silver

In [4]:
#Pandas on Snowflake can only use one session, explicitly assign one of the sessions to pandas
pd.session = session

In [4]:
data = pd.read_snowflake('TRACKLINK')

In [5]:
data = data[(data['MODELNAME'] == 'D8T') | (data['MODELNAME'] == 'D6TXW')]
print('This dataset has ' + str(data.shape[0]) + ' rows, and ' + str(data.shape[1]) + ' columns')

This dataset has 3226 rows, and 9 columns


In [6]:
data.columns

Index(['MODELNAME', 'SERIALNUMBER', 'INSPECTIONDATE', 'LINKALLOWABLEWEAR',
       'BUSHINGALLOWABLEWEAR', 'TRACKLINKINSTALLEDDATE',
       'TRACKLINKWORNVALUELEFT', 'TRACKLINKWORNVALUERIGHT',
       'HOURMETERREADING'],
      dtype='object')

In [ ]:
data.head(6)

,MODELNAME,SERIALNUMBER,INSPECTIONDATE,LINKALLOWABLEWEAR,BUSHINGALLOWABLEWEAR,TRACKLINKINSTALLEDDATE,TRACKLINKWORNVALUELEFT,TRACKLINKWORNVALUERIGHT,HOURMETERREADING
0,D6TXW,SKL00358,5/5/2017,lesser,lesser,"""2011-05-09T00:00:00Z""",20,18,0
1,D8T,0J8B02831,2017-08-24T00:00:00-0300,lesser,lesser,"""2011-10-21T00:00:00Z""",38,28,0
2,963D,LCS01949,2017-06-05T18:27:36-0500,lesser,lesser,"""2014-06-03T00:00:00-0500""",86,90,0
3,D9T,RJS01644,2017-02-22T00:00:00Z,greater,greater,"""2016-02-01T00:00:00+0800""",78,69,0
4,D5KLGP,KY200858,2017-02-07T10:42:11-0500,greater,lesser,"""2016-04-29T00:00:00-0400""",4,4,0
5,336DL,WRK00248,2/13/2018,greater,greater,"""""",66,80,0


In [15]:
#change all 0s to 1 in Hour meter reading.
data['HOURMETERREADING']=data['HOURMETERREADING'].replace(0,1)
data.drop(['LINKALLOWABLEWEAR','BUSHINGALLOWABLEWEAR',],axis=1,inplace=True)
data['SERIALNUMBER'] = data['SERIALNUMBER'].str.upper()

In [6]:
conditions1 = data['TRACKLINKWORNVALUELEFT'].isin(['0', '>120', '<0', '<0%', '--', 'Low', 'High', '>120%']) | data['TRACKLINKWORNVALUELEFT'].isna()
conditions2 = data['TRACKLINKWORNVALUERIGHT'].isin(['0', '>120', '<0', '<0%', '--', 'Low', 'High', '>120%']) | data['TRACKLINKWORNVALUERIGHT'].isna()


# Clean the columns
data['TRACKLINKWORNVALUELEFT'] = np.where(conditions1, 130, data['TRACKLINKWORNVALUELEFT'])
data['TRACKLINKWORNVALUERIGHT'] = np.where(conditions2, 130, data['TRACKLINKWORNVALUERIGHT'])

`np.where` implementation may have mismatches with pandas:
Returns a Snowpark pandas object instead of a np array.


In [7]:
data['TRACKLINKWORNVALUELEFT']=data['TRACKLINKWORNVALUELEFT'].astype(int)
data['TRACKLINKWORNVALUERIGHT']=data['TRACKLINKWORNVALUERIGHT'].astype(int)

Snowpark pandas API auto cast all integers to int64


In [8]:
data = data.drop(data[(data['TRACKLINKWORNVALUELEFT'] == 130) & (data['TRACKLINKWORNVALUERIGHT'] == 130)].index)

The current operation leads to materialization and can be slow if the data is large!


In [9]:
data_k = data

In [10]:
#data = data.drop(data[data.tracklinkinstalleddate == '""'].index)
data['TRACKLINKINSTALLEDDATE'] = data['TRACKLINKINSTALLEDDATE'].str.replace('"', '').replace('"','').replace('','2003-07-21')

In [11]:
#clean up date, put NaT for invalids, remove time component by converting to date, then convert back to datetime
data['TRACKLINKINSTALLEDDATE'] = pd.to_datetime(data['TRACKLINKINSTALLEDDATE'],errors='coerce')
data['TRACKLINKINSTALLEDDATE'] = data['TRACKLINKINSTALLEDDATE'].dt.date
data['TRACKLINKINSTALLEDDATE'] = pd.to_datetime(data['TRACKLINKINSTALLEDDATE'])

`to_datetime` implementation may have mismatches with pandas:
Snowflake automatic format detection is used when a format is not provided.In this case Snowflake's auto format may yield different result values compared to pandas.See https://docs.snowflake.com/en/sql-reference/date-time-input-output#supported-formats-for-auto-detection for details.


In [12]:
data['INSPECTIONDATE'] = pd.to_datetime(data['INSPECTIONDATE'],errors='coerce')
data['INSPECTIONDATE'] = data['INSPECTIONDATE'].dt.date
data['INSPECTIONDATE'] = pd.to_datetime(data['INSPECTIONDATE'])

In [13]:
data['WORNOUTVALUEMAX']=data[["TRACKLINKWORNVALUELEFT", "TRACKLINKWORNVALUERIGHT"]].max(axis=1)

In [20]:
data['WORNOUTVALUEMAX']=data['WORNOUTVALUEMAX'].astype(int)

In [18]:
data.tail()

,MODELNAME,SERIALNUMBER,INSPECTIONDATE,TRACKLINKINSTALLEDDATE,TRACKLINKWORNVALUELEFT,TRACKLINKWORNVALUERIGHT,HOURMETERREADING,WORNOUTVALUEMAX
38883,D8T,VIRADOR DE VAGAO 04,2018-05-16,2003-07-21,10,10,1,10
38903,D8T,VIRADOR DE VAG��O,2018-05-21,2003-07-21,3,130,1,130
38907,D8T,VIRADOR DE VAG��O VV01,2018-05-30,2003-07-21,22,18,1,22
38911,D8T,VIRADOR DE VAG��O VV01,2018-05-30,2003-07-21,8,4,1,8
38925,D8T,VIRADOR E VAG̥ES,2018-05-21,2003-07-21,24,130,1,130


In [21]:
data.to_snowflake("snowpark_db.cat_silver.tracklink", if_exists="replace", index=False )

In [22]:
#Grouped Data Frame
data_grp = data.groupby(['SERIALNUMBER','INSPECTIONDATE'], as_index=False).WORNOUTVALUEMAX.max()
#data1.drop('inspectiondate',axis=1,inplace=True)

In [23]:
data_grp.to_snowflake("snowpark_db.cat_silver.tracklink_grp", if_exists="replace", index=False )